#  프롬프트 엔지니어링 - Chain-of-Thought (CoT) 등 고급 기법 

### 학습 목표
1. Zero-shot, Few-shot, CoT 프롬프팅의 차이점과 적용 시기를 이해한다
2. Self-Consistency, PAL, Reflexion 고급 기법을 학습한다
3. 다양한 LLM 모델(OpenAI, Ollama)의 추론 능력을 비교한다
4. 복잡한 논리적 추론 문제에 적절한 프롬프팅 기법을 적용할 수 있다

---

## 환경 설정 및 준비

`(1) Env 환경변수`

**.env 파일 설정 예시:**
```
OPENAI_API_KEY=your_api_key_here
```

**Ollama 모델 사전 설치:**
```bash
ollama pull phi3:mini
ollama pull gemma2:2b
ollama pull deepseek-r1:7b
```

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-nano',
    temperature=0.3,  # 응답의 무작위성 조절 (0: 결정적, 1: 창의적)
    top_p=0.9,        # 누적 확률 기반 토큰 선택
)

In [ ]:
from langchain_ollama import ChatOllama

ollama = ChatOllama(
    model='phi3:mini',
    temperature=0.3,  # 응답의 무작위성 조절 (0: 결정적, 1: 창의적)
    top_p=0.9,        # 누적 확률 기반 토큰 선택
)

---

## **Chain of Thought (CoT)**

* Chain of Thought는 AI 모델이 복잡한 문제를 해결할 때 각 단계별 사고 과정을 명시적으로 보여주도록 하는 프롬프팅 기법으로, 이를 통해 모델의 추론 과정을 투명하게 확인할 수 있고 더 정확한 결과를 도출할 수 있습니다.

* 이 방식은 특히 수학 문제 풀이, 논리적 추론이 필요한 과제, 복잡한 의사결정 과정에서 매우 효과적이며, 모델이 중간 단계에서 발생할 수 있는 오류를 스스로 발견하고 수정할 수 있게 합니다.

* CoT의 주요 장점은 문제 해결 과정의 투명성을 높이고 최종 답변의 신뢰성을 향상시킬 수 있다는 것이지만, 출력이 길어지고 계산 비용이 증가할 수 있다는 단점도 존재합니다.


`(1) Zero-shot 프롬프팅`

   - 가장 단순한 형태의 프롬프팅
   - 예시나 단계별 설명 없이 직접 답을 출력
   - 속도가 빠르고 메모리 사용량이 적은 편
   - 단순한 문제에 적합

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
zero_shot_template = """
다음 문제를 해결하시오:

문제: {question}

답안:
"""

zero_shot_prompt = PromptTemplate(
    input_variables=["question"],
    template=zero_shot_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# 프롬프트 출력
print(zero_shot_prompt.format(question=question))

In [ ]:
# OpenAI gpt-4.1-nano 모델로 답안 생성

zero_shot_chain = zero_shot_prompt | llm 

answer = zero_shot_chain.invoke({"question": question})

print(answer.content)

In [ ]:
# Ollama Phi3:mini 모델로 답안 생성

zero_shot_chain = zero_shot_prompt | ollama

answer = zero_shot_chain.invoke({"question": question})

print(answer.content)

`(2) One-shot/Few-shot 프롬프팅`

   - 하나 이상의 예시를 통해 문제 해결 방식을 제시 
   - 유사한 예시를 통해 학습 효과를 기대
   - Zero-shot보다 더 정확한 결과를 얻을 수 있음
   - 중간 복잡도의 문제에 적합

   - 논문: https://arxiv.org/abs/2005.14165

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
one_shot_template = """
다음은 수학 문제를 해결하는 예시입니다:

예시 문제: 한 학급에 30명의 학생이 있습니다. 이 중 40%가 남학생이라면, 여학생은 몇 명인가요?

예시 풀이:
1) 먼저 남학생 수를 계산합니다:
   - 전체 학생의 40% = 30 x 0.4 = 12명이 남학생

2) 여학생 수를 계산합니다:
   - 전체 학생 수 - 남학생 수 = 30 - 12 = 18명이 여학생

따라서 여학생은 18명입니다.

이제 아래 문제를 같은 방식으로 해결하시오:

새로운 문제: {question}

답안:
"""

one_shot_prompt = PromptTemplate(
   input_variables=["question"],
   template=one_shot_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# 프롬프트 출력
print(one_shot_prompt.format(question=question))

In [ ]:
# OpenAI gpt-4.1-nano 모델로 답안 생성

one_shot_chain = one_shot_prompt | llm 

answer = one_shot_chain.invoke({"question": question})

print(answer.content)

In [ ]:
# Ollama Phi3:mini 모델로 답안 생성

one_shot_chain = one_shot_prompt | ollama

answer = one_shot_chain.invoke({"question": question})

print(answer.content)

`(3) Chain of Thought(CoT) 프롬프팅`

   - 가장 체계적인 문제 해결 방식을 제공
   - 명시적인 단계별 추론 과정을 포함
   - 복잡한 문제 해결에 가장 적합

   - 논문: https://arxiv.org/abs/2201.11903

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
cot_template = """
다음 문제를 논리적 단계에 따라 해결하시오:
문제: {question}

해결 과정:
1단계: 문제 이해하기
- 주어진 정보 파악
- 구해야 할 것 정리

2단계: 해결 방법 계획
- 사용할 수 있는 전략 검토
- 최적의 방법 선택

3단계: 계획 실행
- 선택한 방법 적용
- 중간 결과 확인

4단계: 검토
- 답안 확인
- 다른 방법 가능성 검토

답안:
"""

cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# 프롬프트 출력
print(cot_prompt.format(question=question))

In [ ]:
# OpenAI gpt-4.1-nano 모델로 답안 생성

cot_chain = cot_prompt | llm 

answer = cot_chain.invoke({"question": question})

print(answer.content)

In [ ]:
# Ollama Phi3:mini 모델로 답안 생성

cot_chain = cot_prompt | ollama

answer = cot_chain.invoke({"question": question})

print(answer.content)

In [ ]:
# 다른 오픈소스 모델로 테스트
from langchain_ollama import ChatOllama

gemma = ChatOllama(
    model='gemma2:2b',
    temperature=0.3,  # 응답의 무작위성 조절 (0: 결정적, 1: 창의적)
    top_p=0.9,        # 누적 확률 기반 토큰 선택
)

deepseek = ChatOllama(
    model='deepseek-r1:7b',
    temperature=0.3,  # 응답의 무작위성 조절 (0: 결정적, 1: 창의적)
    top_p=0.9,        # 누적 확률 기반 토큰 선택
)

In [ ]:
# GEMMA2 모델로 답안 생성

cot_chain = cot_prompt | gemma

answer = cot_chain.invoke({"question": question})

print(answer.content)

In [ ]:
# Ollama DeepSeek 추론 모델로 답안 생성 (답변을 생성하기 위해 생각을 먼저 하는 모델로 zero-shot 적용)

zero_shot_chain = zero_shot_prompt | deepseek

answer = zero_shot_chain.invoke({"question": question})

print(answer.content)

---

### **[실습]** 다음 논리적 추론 문제를 gemma2:2b 모델을 사용하여 3가지 유형의 프롬프트를 작성하여 해결하고, 그 성능을 비교합니다. 

(정답: 7번 이동)

In [ ]:
question = """
농부가 늑대, 양, 양배추를 데리고 강을 건너야 함

제약조건:
1. 농부가 없을 때 늑대와 양이 같이 있으면 늑대가 양을 잡아먹음
2. 농부가 없을 때 양과 양배추가 같이 있으면 양이 양배추를 먹어버림
3. 보트에는 농부와 한 물건만 실을 수 있음

모두 안전하게 건너는데 몇 번 이동이 필요할까요. 숫자로 답변하세요.
"""

In [ ]:
# [실습 1-1] Zero-shot 방식으로 답안 생성
# TODO: zero_shot_template을 참고하여 프롬프트 작성
# TODO: ollama 모델(gemma2:2b)과 체인을 구성
# TODO: question 변수로 invoke하여 답안 출력
# 힌트: 앞의 코드 구조를 참고하세요

# 여기에 코드를 작성하세요

In [ ]:
# [실습 1-2] One-shot 방식으로 답안 생성
# TODO: one_shot_template을 참고하여 예시를 포함한 프롬프트 작성
# 힌트: 늑대-양-양배추 문제와 유사한 간단한 예시를 작성해보세요

# 여기에 코드를 작성하세요

In [ ]:
# [실습 1-3] CoT 방식으로 답안 생성
# TODO: 단계별 추론 과정을 명시하는 프롬프트 작성
# 검증: 최종 답이 7번 이동인지 확인하세요

# 여기에 코드를 작성하세요

---

## **Self-Consistency**

* Self-Consistency는 AI 모델에게 하나의 문제에 대해 다양한 접근 방식으로 해결하도록 요청하는 기법으로, 여러 경로를 통해 도출된 결과들의 일관성을 확인함으로써 답변의 신뢰성을 높입니다.

* 이 방법은 특히 수학 문제나 논리적 추론이 필요한 과제에서 효과적이며, 서로 다른 방법으로 도출된 결과가 일치하는지 검증함으로써 오류 가능성을 최소화할 수 있습니다.

* Self-Consistency의 장점은 답변의 정확성을 높일 수 있다는 것이지만, 여러 번의 계산과 추론이 필요하므로 처리 시간이 길어지고 컴퓨팅 리소스 사용량이 증가한다는 단점도 존재합니다.

* 또한 이 기법은 Chain of Thought (CoT) 프롬프팅과 결합하여 사용할 경우 더욱 강력한 효과를 발휘할 수 있습니다.

- 논문: https://arxiv.org/abs/2203.11171


In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
self_consistency_template = """
다음 문제를 세 가지 다른 방법으로 해결하시오:

문제: {question}

세 가지 풀이 방법:
1) 직접 계산 방법:
   - 주어진 숫자를 직접 계산

2) 비율 활용 방법:
   - 전체에 대한 비율로 계산

3) 단계별 분해 방법:
   - 문제를 작은 부분으로 나누어 계산

각 방법의 답안을 제시하고, 결과가 일치하는지 확인하시오.

답안:
"""

self_consistency_prompt = PromptTemplate(
   input_variables=["question"],
   template=self_consistency_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# OpenAI gpt-4.1-nano 모델로 답안 생성
self_consistency_chain = self_consistency_prompt | llm 
answer = self_consistency_chain.invoke({"question": question})

print(answer.content)

---

## **Program-Aided Language (PAL)**

* PAL은 자연어 문제를 프로그래밍적 사고방식으로 접근하도록 하는 기법으로, 복잡한 문제를 코드나 의사코드 형태로 분해하여 해결하는 방식입니다. 이를 통해 문제 해결 과정을 더욱 구조화하고 체계적으로 만들 수 있습니다.

* 이 접근 방식의 큰 장점은 프로그래밍 언어의 정확성과 논리성을 활용하여 모호함을 줄이고, 각 단계를 명확하게 정의할 수 있다는 것입니다. 특히 수학적 계산, 데이터 처리, 알고리즘적 문제 해결에서 뛰어난 성능을 보입니다.

* PAL의 특징적인 점은 실제 실행 가능한 코드를 생성할 수 있다는 것으로, 이는 결과의 검증이 용이하고 필요한 경우 수정이나 최적화가 가능하다는 장점이 있습니다. 

- 논문: https://arxiv.org/pdf/2211.10435


In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
pal_template = """
다음 문제를 Python 프로그래밍 방식으로 해결하시오:

문제: {question}

# 문제 해결을 위한 Python 스타일 의사코드:
def solve_problem():
    # 1. 변수 정의
    # - 주어진 값들을 변수로 저장
    
    # 2. 계산 과정
    # - 필요한 계산을 단계별로 수행
    
    # 3. 결과 반환
    # - 최종 결과 계산 및 반환
    
답안:
"""

pal_prompt = PromptTemplate(
    input_variables=["question"],
    template=pal_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# OpenAI gpt-4.1-nano 모델로 답안 생성
pal_chain = pal_prompt | llm 
answer = pal_chain.invoke({"question": question})

print(answer.content)

---

## **Reflexion**

* Reflexion은 AI가 자신의 이전 답변을 스스로 검토하고 평가하여 개선하는 메타인지적 프롬프팅 기법으로, 이를 통해 응답의 질을 점진적으로 향상시킬 수 있습니다.

* 이 방법은 AI가 자신의 답변에서 부족한 점, 오류, 또는 개선이 필요한 부분을 스스로 찾아내고 수정하도록 함으로써, 더 정확하고 완성도 높은 답변을 도출할 수 있게 합니다. 특히 복잡한 분석이나 창의적인 작업에서 효과적입니다.

* Reflexion의 강점은 AI가 자기 평가를 통해 지속적으로 개선된 결과물을 제공할 수 있다는 것이지만, 여러 번의 반복적인 검토와 수정 과정이 필요하므로 시간과 컴퓨팅 자원이 더 많이 소요될 수 있다는 제한점이 있습니다.

* 이 기법은 특히 글쓰기, 코드 리뷰, 분석 리포트 작성 등 높은 품질의 출력이 요구되는 작업에서 매우 유용하게 활용될 수 있습니다.

- 논문: https://arxiv.org/abs/2303.11366

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
reflexion_template = """
다음 문제에 대해 단계적으로 해결하여 초기 답안을 작성하고, 자체 평가 후 개선하시오:

문제: {question}

1단계: 초기 답안
---
[여기에 첫 번째 답안 작성]

2단계: 자체 평가
---
- 정확성 검토
- 논리적 오류 확인
- 설명의 명확성 평가
- 개선이 필요한 부분 식별

3단계: 개선된 답안
---
[평가를 바탕으로 개선된 답안 작성]

답안:
"""

reflexion_prompt = PromptTemplate(
    input_variables=["question"],
    template=reflexion_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# OpenAI gpt-4.1-nano 모델로 답안 생성
reflexion_chain = reflexion_prompt | llm  # 이미 정의된 llm 재사용
answer = reflexion_chain.invoke({"question": question})

print(answer.content)

### **[실습 2]** [실습 1]의 논리적 추론 문제를 gemma2:2b 모델을 사용하여 3가지 유형의 프롬프트를 작성하여 해결하고, 그 성능을 비교합니다. 

(정답: 7번 이동)

In [ ]:
# [실습 2-1] Self-Consistency 방식으로 답안 생성
# TODO: self_consistency_template을 참고하여 프롬프트 작성
# TODO: gemma2:2b 모델과 체인을 구성 (gemma 변수 사용)
# TODO: question 변수로 invoke하여 답안 출력
# 힌트: 앞의 코드 구조를 참고하세요

# 여기에 코드를 작성하세요

In [ ]:
# [실습 2-2] Program-Aided Language 방식으로 답안 생성
# TODO: pal_template을 참고하여 프로그래밍적 프롬프트 작성
# TODO: gemma2:2b 모델과 체인을 구성
# 힌트: Python 의사코드 형태로 문제 해결 과정을 구조화하세요

# 여기에 코드를 작성하세요

In [ ]:
# [실습 2-3] Reflexion 방식으로 답안 생성
# TODO: reflexion_template을 참고하여 자체 평가 프롬프트 작성
# TODO: gemma2:2b 모델과 체인을 구성
# 검증: 초기 답안과 개선된 답안의 차이를 비교하세요

# 여기에 코드를 작성하세요